In [ ]:
import os
import obspy
import numpy as np
import pandas as pd
import random as random
from tensorflow.keras.models import load_model
%matplotlib inline
os.getcwd()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
signalmodel = load_model('signalmodel1')
locmodel = load_model('locmodel1')

In [ ]:
def detector_predict(trace):
    
    ## Things to Specify -- Ensure that they align with trained model
    # For rolling averages
    roll_short = 25
    roll_long = 50

    # For location segments
    window_step = 30
    window_size = 30
    
    ## Reading in Trace and Splitting Channels
    sig_trace = trace.normalize()
    
    samp_rate = sig_trace.stats.sampling_rate
    start_time = sig_trace.stats.starttime
    
    sigs = sig_trace.data
    
    sig_df = pd.DataFrame(sigs, columns = ["trace"])
    
    sigfeatures = []
    
    ## Calculating various features needed for the signal model
    tr = sig_df["trace"]
    mag = abs(tr)
    d = {"trace":tr, "magnitude":mag}
    temp_df = pd.DataFrame(data = d)
    
    temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
    temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
    temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
    temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
    temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()

    temp_df.dropna(inplace = True)
    sigfeatures.append(temp_df.values)
    
    ## Predicting whether or not there is a pwave in the trace
    is_sig = signalmodel.predict(np.array(sigfeatures))
    print(is_sig)
    
    if is_sig < 0.5:
        return None
    
    else:
        locfeatures = []

        ## If there is a pwave in the trace, continue to find location of pwave
        start_ind = 0
        end_ind = start_ind + window_size

        while end_ind < (1000 - roll_long):
            trwindow = temp_df["trace"].iloc[start_ind:end_ind]
            magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
            ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
            stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
            ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

            window_data = {"trace": trwindow, "magnitude": magwindow,
                            "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}

            window_df = pd.DataFrame(data = window_data)

            locfeatures.append(window_df.values)

            start_ind += window_step
            end_ind = start_ind + window_size

        prob_vec = locmodel.predict(np.array(locfeatures))
        p_segment = np.where(prob_vec == max(prob_vec))[0][0]

        tick_delta = (p_segment + 0.5)*window_size
        time_delta = tick_delta/samp_rate
        p_time = start_time + time_delta

        return p_time  

In [ ]:
os.chdir('lse_challenge/data/signal')
signal_files = os.listdir()

In [ ]:
signal_file = random.sample(signal_files, 1)
stream = obspy.read(signal_file[0])
tr = stream[0]
cut = int((random.random()*20+1)*32.5)
tr.data = tr.data[cut:1000+cut]
correct_answer = tr.stats.starttime + (1000-cut)/tr.stats.sampling_rate
print(correct_answer)

In [ ]:
prediction = detector_predict(tr)
if(prediction):
    print(abs(correct_answer-prediction))
else:
    print(prediction)